<a href="https://colab.research.google.com/github/yananma/5_programs_per_day/blob/master/02153.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 6.4 循环神经网络的从零开始实现

In [0]:
import torch 
import math 
import numpy as np 
from torch import nn, optim 
import torch.nn.functional as F 
import d2l 
import time 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
!mkdir ../../data

In [8]:
!git clone https://github.com/ShusenTang/Dive-into-DL-PyTorch.git

Cloning into 'Dive-into-DL-PyTorch'...
remote: Enumerating objects: 1692, done.
remote: Total 1692 (delta 0), reused 0 (delta 0), pack-reused 1692
Receiving objects: 100% (1692/1692), 25.29 MiB | 16.89 MiB/s, done.
Resolving deltas: 100% (975/975), done.


In [0]:
!cp Dive-into-DL-PyTorch/data/jaychou_lyrics.txt.zip ../../data

In [0]:
(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics()

### 6.4.1 one-hot 向量

In [11]:
def one_hot(x, n_class, dtype=torch.float32):
    x = x.long()
    res = torch.zeros(x.shape[0], n_class, dtype=dtype, device=x.device)
    res.scatter_(1, x.view(-1, 1), 1)
    return res 

x = torch.tensor([0, 2])
one_hot(x, vocab_size)

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.]])

In [12]:
def to_onehot(X, n_class):
    return [one_hot(X[:, i], n_class) for i in range(X.shape[1])]

X = torch.arange(10).view(2, 5)
inputs = to_onehot(X, vocab_size)
len(inputs), inputs[0].shape

(5, torch.Size([2, 1027]))

### 6.4.2 初始化模型参数

In [23]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size 
print('will use', device)

def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype=torch.float32)
        return torch.nn.Parameter(ts, requires_grad=True)

    W_xh = _one((num_inputs, num_hiddens))
    W_hh = _one((num_hiddens, num_hiddens))
    b_h = torch.nn.Parameter(torch.zeros(num_hiddens, device=device, requires_grad=True))
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device, requires_grad=True))
    return nn.ParameterList([W_xh, W_hh, b_h, W_hq, b_q])

will use cuda


### 6.4.3 定义模型

In [0]:
def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), )

In [0]:
def rnn(inputs, state, params):
    W_xh, W_hh, b_h, W_hq, b_q = params 
    H, = state 
    outputs = []
    for X in inputs:
        H = torch.tanh(torch.matmul(X, W_xh) + torch.matmul(H, W_hh) + b_h)
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H, )

In [27]:
state = init_rnn_state(X.shape[0], num_hiddens, device)
inputs = to_onehot(X.to(device), vocab_size)
params = get_params()
outputs, state_new = rnn(inputs, state, params)
len(outputs), outputs[0].shape, state_new[0].shape

(5, torch.Size([2, 1027]), torch.Size([2, 256]))

### 6.4.4 定义预测函数

In [0]:
def predict_rnn(prefix, num_chars, rnn, params, init_rnn_state, 
        num_hiddens, vocab_size, device, idx_to_char, char_to_idx):
    state = init_rnn_state(1, num_hiddens, device) 
    output = [char_to_idx[prefix[0]]]
    for t in range(num_chars + len(prefix) - 1):
        X = to_onehot(torch.tensor([[output[-1]]], device=device), vocab_size)
        (Y, state) = rnn(X, state, params)
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(int(Y[0].argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])

In [29]:
predict_rnn('分开', 10, rnn, params, init_rnn_state, num_hiddens, vocab_size, device, idx_to_char, char_to_idx)

'分开吴苍狠鼠苍狠鼠苍狠鼠'

### 6.4.5 裁剪梯度

In [0]:
def grad_clipping(params, theta, device):
    norm = torch.tensor([0.0], device=device)
    for param in params:
        norm += (param.grad.data ** 2).sum()
    norm = norm.sqrt().item()
    if norm > theta:
        for param in params:
            param.grad.data *= (theta / norm)

### 6.4.7 定义模型训练函数

In [0]:
def train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens, vocab_size, device, corpus_indices, idx_to_char, 
            char_to_idx, is_random_iter, num_epochs, num_steps, lr, clipping_theta, batch_size, pred_peroid, 
            pred_len, prefixes):
    if is_random_iter:
        data_iter_fn = d2l.data_iter_random 
    else:
        data_iter_fn = d2l.data_iter_consecutive 
    params = get_params()
    loss = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        if not is_random_iter:
            state = init_rnn_state(batch_size, num_hiddens, device)
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_fn(corpus_indices, batch_size, num_steps, device)
        for X, Y in data_iter:
            if is_random_iter:
                state = init_rnn_state(batch_size, num_hiddens, device)
            else:
                for s in state:
                    s.detach_()

            inputs = to_onehot(X, vocab_size)
            (outputs, state) = rnn(inputs, state, params)
            outputs = torch.cat(outputs, dim=0)
            y = torch.transpose(Y, 0, 1).contiguous().view(-1)
            l = loss(outputs, y.long())

            if params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            l.backward()
            grad_clipping(params, clipping_theta, device)
            d2l.sgd(params, lr, 1)
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]

        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' 
                % (epoch + 1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn(prefix, pred_len, rnn, params, init_rnn_state, 
                   num_hiddens, vocab_size, device, idx_to_char, char_to_idx))

### 6.4.8 训练模型并创作歌词

In [0]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 250, 35, 32, 1e2, 1e-2 
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']

In [44]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens, vocab_size, device, corpus_indices, idx_to_char, 
        char_to_idx, True, num_epochs, num_steps, lr, clipping_theta, batch_size, pred_period, pred_len, prefixes)

epoch 50, perplexity 68.871115, time 0.09 sec
 - 分开 我想要这 你使了 别怪我 娘子就 一直么 我想要的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯
 - 不分开 我想要再 你小了 别怪我 泪不 让我的红  谁的让我 狂的可爱女人 一坏的让我疯狂的可爱女人 坏坏
epoch 100, perplexity 10.213086, time 0.09 sec
 - 分开 一直两它一步四步 连成线背著背 我怀在我生朋友样外加找 你十的让我面红 可小就耳濡路染 我想要你的
 - 不分开吗 我不要和你离着我的手不 铺原的外板人  爱穿了的话剩 像 在我胸口睡久 像这样的生活 我的你 你
epoch 150, perplexity 2.783084, time 0.09 sec
 - 分开 一步两它开 谁让它停留的 为什么我女朋友场外加油 你却还让我出糗 从小就耳濡路染 什么刀枪了棍棒 
 - 不分开扫 然后将过了 慢慢温在 全暖了空屋 白色蜡烛 温暖了空屋 白色蜡烛 温暖了空屋 白色蜡烛 温暖了空
epoch 200, perplexity 1.637130, time 0.09 sec
 - 分开 一步在它因不达米亚平  养的我猫坦口让记 不有再这 你想我有多的菸 一小恩怨 在上耿中  一场梦 
 - 不分开期 然后将过去 慢慢温习 全家了空出 白色蜡烛 温暖了空屋 白色蜡烛 温暖了空屋 白色蜡烛 温暖了空
epoch 250, perplexity 1.302495, time 0.09 sec
 - 分开 一直令它留 谁让它停留的 为什么我女朋友场外加油 你却还让我出糗 可小就迷了路怎么找也找不着 心血
 - 不分开扫 我叫你爸 你打我妈 这样对吗干嘛这样 何必让酒牵鼻子走 瞎 说说你打单车一个 想想你说样来我单妈


In [45]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens, vocab_size, device, corpus_indices, idx_to_char, 
        char_to_idx, False, num_epochs, num_steps, lr, clipping_theta, batch_size, pred_period, pred_len, prefixes)

epoch 50, perplexity 60.996227, time 0.09 sec
 - 分开 我想要再 你有 我想要再不 我不 再不 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想
 - 不分开 我想要你的溪 我不要这 我有了空 你不了 别怪我 一颗我 一颗我 一颗我 一颗我 一颗我 一颗我 
epoch 100, perplexity 7.106263, time 0.09 sec
 - 分开 一场的假旧在 帅呆了我 全场了人 习果用直在过 哼哼哈兮 快使用双截棍 哼哼哈兮 快使用双截棍 哼
 - 不分开觉 你已经抽 我谁了有节 我一定空 装我了外的溪边 我默店的手色在西元前 深埋在美索不达米亚平  姥
epoch 150, perplexity 2.100979, time 0.10 sec
 - 分开 一养堂 你在我 说你球 干沉空 停子病一信代 唱有人看着 是让它人鸦的你爱下 在够翻文引刻 所有在
 - 不分开觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后兮 快使用双截棍 哼哼哈兮 
epoch 200, perplexity 1.299248, time 0.09 sec
 - 分开 是你已 你打堂枪 回止看 在一句珍停留 一直在停留 谁让它 一句惹毛三会 不着人停留 谁让它 一烧
 - 不分开觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生
epoch 250, perplexity 1.160078, time 0.09 sec
 - 分开 问候就这样打是开 回到儿回爱打我妈妈 难道你手不会痛吗 其实我回家就想要阻止一切 让家庭回到过去甜
 - 不分开觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生
